In [1]:
import pprint

import numpy as np
import pandas as pd
import xarray as xr
import lightgbm as lgb

from otb.tasks import TaskApi
from otb.benchmark.models import (
    PersistanceRegressionModel,
    PersistanceForecastingModel,
    AWTModel,
    MacroMeterologicalModel,
    OffshoreMacroMeterologicalModel
)

In [2]:
usna_sm = xr.load_dataset("otb/data/usna_cn2_sm/usna_cn2_sm.nc")
usna_lg = xr.load_dataset("otb/data/usna_cn2_lg/usna_cn2_lg.nc")
mlo_cn2 = xr.load_dataset("otb/data/mlo_cn2/mlo_cn2.nc")

In [3]:
usna_sm

<xarray.Dataset>
Dimensions:    (time: 22081)
Coordinates:
  * time       (time) datetime64[ns] 2021-06-01 ... 2021-09-01
    lat        float64 38.98
    lon        float64 -76.48
    alt        float64 3.0
Data variables:
    Cn2_3m     (time) float64 2.432e-15 4.29e-15 ... 6.048e-15 7.054e-15
    T_5m       (time) float64 16.6 16.5 16.5 16.6 16.3 ... 26.4 26.3 26.1 26.1
    P_10m      (time) float64 1.025e+03 1.025e+03 ... 1.008e+03 1.008e+03
    RH_3m      (time) float64 88.5 87.35 87.45 88.15 ... 81.0 80.9 80.35 80.0
    Spd_10m    (time) float64 0.4 0.1 0.5 0.6 0.8 1.0 ... 1.6 1.5 2.0 1.8 1.5
    Dir_10m    (time) float64 179.0 206.0 191.0 284.0 ... 37.0 37.0 54.0 65.0
    Rad_1m     (time) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    T_0m       (time) float64 20.1 20.1 20.1 20.2 20.2 ... 29.5 29.4 29.4 29.4
    base_time  datetime64[ns] 2021-06-01
Attributes:
    description:  Scintillometer, meterological, and oceanographic data from ...
    authors:      Christopher Jellen, Charles Nelson, John Burkhardt, and Cod...
    history:      Created from raw data on 2023-08-25 using package `argcli`,...

In [4]:
usna_lg

<xarray.Dataset>
Dimensions:        (time: 1291225)
Coordinates:
  * time           (time) datetime64[ns] 2020-01-01 ... 2022-07-14
    lat            float64 38.98
    lon            float64 -76.48
    alt            float64 3.0
Data variables:
    Dir_3m         (time) object 'nan' 'nan' 'nan' 'nan' ... 'nan' 'nan' 'SW'
    temporal_hour  (time) float64 -9.388 -9.367 -9.346 ... 14.81 14.83 -4.81
    Cn2_3m         (time) float64 nan 1.42e-15 1.263e-15 ... 2.419e-15 2.22e-15
    T_3m           (time) float64 6.7 6.7 6.7 6.7 6.7 ... 27.46 27.44 27.42 27.4
    P_3m           (time) float64 1.008e+03 1.008e+03 ... 1.011e+03 1.011e+03
    RH_3m          (time) float64 57.0 57.0 57.0 57.0 ... 80.3 80.2 80.1 80.0
    Spd_3m         (time) float64 2.7 2.7 2.7 2.7 2.7 ... 1.18 1.22 1.26 1.3
    Rad_1m         (time) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    T_0m           (time) float64 6.4 6.4 6.4 6.4 6.4 ... nan nan nan nan nan
    base_time      datetime64[ns] 2020-01-01
Attributes:
    description:  Scintillometer, meterological, and oceanographic data from ...
    authors:      Christopher Jellen, Charles Nelson, John Burkhardt, and Cod...
    history:      Created from raw data on 2023-08-25 using package `argcli`,...

In [5]:
%matplotlib inline

In [6]:
pprinter = pprint.PrettyPrinter(indent=4, width=120, compact=True)

The optical turbulence benchmark package provides a set of tasks to be used for the evaluation of optical turbulence prediction methods.

The package includes both regression tasks, and forecasting tasks. Each regression task assesses a given model's ability to predict the optical turbulence strength (as measured by $C_n^2$) at a given location, given a set of meterological parameters. Each forecasting task assesses a given model's ability to predict the optical turbulence strength some number of time steps in the future given a set of prior meterological parameters and previous measurements of the optical turbulence strength at that location.

Each task contains a dataset and a set of metrics.

#### Datasets

Datasets contain some number of timestamped observations of the optical turbulence strength alongside a set of potentially relevant meterological and oceanographic parameters. Datasets may or may not contain missing measurements, and strive to conform to the [NetCDF](https://docs.unidata.ucar.edu/netcdf-c/current/index.html) Climate and Forecast (CF) [Metadata Conventions](http://cfconventions.org/).

Under the hood, datasets are stored as [xarray](http://xarray.pydata.org/en/stable/) `Dataset` objects. They are serialized to disk as [NetCDF](https://docs.unidata.ucar.edu/netcdf-c/current/index.html) files. Datasets are shared between one or more tasks. For example, the `mlo_cn2` dataset is used by both the `mlo_cn2` regression task and the `mlo_cn2_forecast` forecasting task.

The train, test, and validation splits are defined by the task using fixed indices. Tasks also define data processing pipelines that are applied to the data before it is used for training, testing, or validation. This can include common techniques such as removing rows with missing measurements, or taking the log of the optical turbulence strength. Finally, tasks define the set of features which are unavailable for training. Again taking the `mlo_cn2` tasks as an example, the target feature is the optical turbulence strength at a height of 15 \[m\]; the unavailable features are the optical turbulence strength at other heights are assumed as unavailable for training or inference in the regression task.

Tasks evaluate the performance of a model on the test and validation splits using the metrics defined by the task.

#### Metrics

The metrics are used to evaluate the performance of a model or prediction method on the data. Metrics, in the context of `otb` tasks, allow for rigorous comparison of different models and prediction methods. Metrics are defined by the task, and are evaluated on the test and validation splits of the dataset.

Many tasks use standard error metrics including:
* mean absolute error (MAE)
* explained variance score (EVS or $R^2$)
* root mean squared error (RMSE)
* mean absolute percentage error (MAPE)

All regression and forecasting tasks include some baseline models which can be applied to the prediction problem. Each task's metrics are evaluated on the baseline models and the results are stored in a shared `experiments.json` file. This allows for easy comparison of different models and prediction methods. When developing a new model or prediction method, it is recommended to compare the performance of the new method to the baseline models. After the new method has been evaluated, it can be programmatically added to the `experiments.json` file for future comparison using the task's interface.

#### Example: (`mlo_cn2`) regression task, without missing values

```
{
    'description': 'Regression task for MLO Cn2 data, ...',
    'description_long': 'This dataset evaluates ...',
    'dropna': True,
    'ds_name': 'mlo_cn2',
    'eval_metrics': ['root_mean_square_error', 'r2_score', 'mean_absolute_error', 'mean_absolute_percentage_error'],
    'log_transform': True,
    'obs_lat': 19.53,
    'obs_lon': -155.57,
    'obs_tz': 'US/Hawaii',
    'remove': ['base_time', 'Cn2_6m', 'Cn2_15m', 'Cn2_25m'],
    'target': 'Cn2_15m',
    'val_idx': ['8367:10367'],
    'train_idx': ['0:8367'],
    'test_idx': ['10367:13943']
}
```

### load the tasks

The `TaskApi` is the main entry point for the OTB API.

In [7]:
task_api = TaskApi()

The `TaskApi` provides access to the tasks, which in turn enable access to training, test, and validation data, benchmarking metrics, and evaluation of new prediction models or methods.

The tasks which are currently supported by the `otb` package are accessible via the `TaskApi`:

In [8]:
task_api.list_tasks()

['regression.mlo_cn2.dropna.Cn2_15m',
 'regression.usna_cn2_lg.full.Cn2_3m',
 'forecasting.usna_cn2_sm.full.Cn2_3m',
 'forecasting.mlo_cn2.dropna.Cn2_15m',
 'regression.usna_cn2_sm.full.Cn2_3m',
 'regression.mlo_cn2.full.Cn2_15m']

As an illustrative example, we can load the `mlo_cn2` regression task with missing values removed and develop a new model for predicting optical turbulence strength.

In [9]:
task = task_api.get_task("regression.mlo_cn2.dropna.Cn2_15m")

The `task` object gives access to the description and associated metadata surrounding the task.

In [10]:
task_info = task.get_info()
pprinter.pprint(task_info)

{   'description': 'Regression task for MLO Cn2 data, where the last 12 days are set aside for evaluation',
    'description_long': 'This dataset evaluates regression approaches for predicting the extent of optical turbulence, '
                        'as measured by Cn2 at an elevation of 15m. Optical turbulence on data collected at the Mauna '
                        'Loa Solar Observatory between 27 July 2006 and 8 August 2006, inclusive, are used to evaluate '
                        'prediction accuracy under the root-mean square error metric.',
    'dropna': True,
    'ds_name': 'mlo_cn2',
    'eval_metrics': ['root_mean_square_error', 'r2_score', 'mean_absolute_error', 'mean_absolute_percentage_error'],
    'log_transform': True,
    'obs_lat': 19.53,
    'obs_lon': -155.57,
    'obs_tz': 'US/Hawaii',
    'remove': ['base_time', 'Cn2_6m', 'Cn2_15m', 'Cn2_25m'],
    'target': 'Cn2_15m',
    'test_idx': ['10367:13943'],
    'train_idx': ['0:8367'],
    'val_idx': ['8367:10367']}


As seen above, the `regression.mlo_cn2.dropna.Cn2_15m` task is focused on predicting the optical turbulence strength at a height of 15 \[m\] at the Mauna Loa Observatory (MLO) in Hawaii. The task uses the `mlo_cn2` dataset, which is a dataset of optical turbulence strength measurements at the MLO. The `task` contains an `obs_tz` attribute which specifies the timezone of the observatory. The latitude and longitude of the observatory are also provided as `obs_lat` and `obs_lon` attributes.

The `task` also contains a `target` attribute which specifies the target feature for the task. The task is focused on predicting the optical turbulence strength at a height of 15 \[m\], and the optical turbulence strength measurements at heights of 6 and 25 \[m\] are assumed to be unavailable for training or inference.

To ensure consistency and robust comparison between modeling approaches, the `train_idx`, `test_idx`, and `val_idx` are fixed for the given task. The `train_idx` and `val_idx` attributes specify the indices of the dataset which are available for model development. The `test_idx` attribute specifies the indices of the dataset which are used to evaluate the model during and compare against existing benchmarks for the task.

The task is evaluated using the root mean squared error (RMSE), explained variance score (EVS), mean absolute error (MAE), and mean absolute percentage error (MAPE) metrics. The task is evaluated on the test and validation splits of the dataset, and the training split is used for training new models.

Get the training data

In [11]:
X_train, y_train = task.get_train_data(data_type="pd")

The `otb` package attempts to make as few assumptions about the model or prediction method's API surface as possible. A major constraint is the assumption that each model is called during evaluation against the validation set in the same form as is returned by the `get_training_data` method with the `data_type` argument set to `pd`.

Models can take many forms, from simple statistical models such as predicting the mean value seen during training, to complex deep learning models. The `otb` package does not attempt to provide a unified API for developing all models, but instead provides a set of tools for evaluating models against the tasks.

Existing statistical and parametric techniques are included under the `otb.benchmark.models` module. These models provide samples of best practices for developing new models for the tasks. An example statistical method which predicts the mean value seen during training is included below.

```python
class MeanRegressionModel:

    def __init__(
        self,
        name: str,
        **kwargs
    ):
        self.name = name
        self.mean = np.nan
    
    def train(self, X: 'pd.DataFrame', y: Union['pd.DataFrame', 'pd.Series', np.ndarray]):
        # maintain the same interface as the other models
        self.mean = np.mean(y)

    def predict(self, X: 'pd.DataFrame'):
        # predict the mean for each entry in X
        return np.full(len(X), self.mean)
```

When evaluated, the `MeanRegressionModel`s performance is measured by calling the `predict` method on the validation data and comparing the results to the ground truth values. The `MeanRegressionModel` has already been evaluated against the metrics defined by the task, and the results are stored in the `experiments.json` file.

We can use the LightGBM library to develop a new model for predicting the optical turbulence strength at a height of 15 \[m\] at MLO to demonstrate this functionality.

Define your model

In [12]:
from otb.benchmark.models.climatology_regression import ClimatologyRegressionModel

model = ClimatologyRegressionModel(name="clim")

In [13]:
from otb.benchmark.bench_runner import run_benchmarks

run_benchmarks()

Running benchmark for forecasting.mlo_cn2.dropna.Cn2_15m...
{   'description': 'Forecasting task for MLO Cn2 data, where the last 12 days are set aside for evaluation',
    'description_long': 'This dataset evaluates forecasting approaches for predicting the extent of optical '
                        'turbulence, as measured by Cn2 at an elevation of 15m. Optical turbulence on data collected '
                        'at the Mauna Loa Solar Observatory between 27 July 2006 and 8 August 2006, inclusive, are '
                        'used to evaluate prediction accuracy for measured Cn2 one observation (6 minutes) in the '
                        'future. The forecasting task makes the last 4 observations of meterological parameters and '
                        'measured Cn2 available as inputs to forecasting models.',
    'dropna': True,
    'ds_name': 'mlo_cn2',
    'eval_metrics': ['root_mean_square_error', 'r2_score', 'mean_absolute_error', 'mean_absolute_percentage_error'],
    '

{'forecasting.mlo_cn2.dropna.Cn2_15m': {'possible_predictions': 2444,
  'mean_window_forecasting': {'root_mean_square_error': {'metric_value': 0.2779041826725006,
    'valid_predictions': 2444},
   'r2_score': {'metric_value': 0.8227610719548378, 'valid_predictions': 2444},
   'mean_absolute_error': {'metric_value': 0.196153461933136,
    'valid_predictions': 2444},
   'mean_absolute_percentage_error': {'metric_value': 0.014085889793932438,
    'valid_predictions': 2444}},
  'linear_forecasting': {'root_mean_square_error': {'metric_value': 0.45668903701202007,
    'valid_predictions': 2444},
   'r2_score': {'metric_value': 0.5213587668339381, 'valid_predictions': 2444},
   'mean_absolute_error': {'metric_value': 0.3222100714029538,
    'valid_predictions': 2444},
   'mean_absolute_percentage_error': {'metric_value': 0.023111269407456055,
    'valid_predictions': 2444}},
  'macro_meterological': {'root_mean_square_error': {'metric_value': 0.7114374938363093,
    'valid_predictions': 809

Once we have defined the model, we can use the training data provided by the task API (`X_train` and `y_train`).


In [14]:
model.train(X_train, y_train)

We can tune hyper parameters using the validation data provided by the task API (`X_val` and `y_val`). In this case, we move directly to evaluation on the test set for our regression task.

Evaluate your model

In [15]:
task.evaluate_model(predict_call=model.predict, x_transforms=None, x_transform_kwargs=None)

{'root_mean_square_error': {'metric_value': 0.5112560391426086,
  'valid_predictions': 2449},
 'r2_score': {'metric_value': 0.401192070766647, 'valid_predictions': 2449},
 'mean_absolute_error': {'metric_value': 0.39026108384132385,
  'valid_predictions': 2449},
 'mean_absolute_percentage_error': {'metric_value': 0.028233831748366356,
  'valid_predictions': 2449}}

The `otb` package includes benchmarks for the models defined in the `otb.benchmark.models` module. Each task can access the evaluation results for the benchmark models using the `get_benchmark_results` method.

Compare against benchmarks

In [16]:
benchmarks = task.get_benchmark_info()

In [13]:
pprinter.pprint(benchmarks)

{   'macro_meterological': {   'mean_absolute_error': {'metric_value': 0.5874813616291615, 'valid_predictions': 813},
                               'mean_absolute_percentage_error': {   'metric_value': 0.04435200340221298,
                                                                     'valid_predictions': 813},
                               'r2_score': {'metric_value': -1.0279853591950694, 'valid_predictions': 813},
                               'root_mean_square_error': {   'metric_value': 0.7138353117427121,
                                                             'valid_predictions': 813}},
    'mean_regression': {   'mean_absolute_error': {'metric_value': 0.5308192372322083, 'valid_predictions': 2449},
                           'mean_absolute_percentage_error': {   'metric_value': 0.038418691605329514,
                                                                 'valid_predictions': 2449},
                           'r2_score': {'metric_value': -0.0014763107411730

We can easily access the top two existing models using the `top_models` method of the `task`.

In [15]:
top_2_models = task.top_models(n=2, metric="mean_absolute_percentage_error")

In [16]:
pprinter.pprint(top_2_models)

{   'mean_regression': {   'mean_absolute_error': {'metric_value': 0.5308192372322083, 'valid_predictions': 2449},
                           'mean_absolute_percentage_error': {   'metric_value': 0.038418691605329514,
                                                                 'valid_predictions': 2449},
                           'r2_score': {'metric_value': -0.001476310741173048, 'valid_predictions': 2449},
                           'root_mean_square_error': {'metric_value': 0.6611728668212891, 'valid_predictions': 2449}},
    'offshore_macro_meterological': {   'mean_absolute_error': {   'metric_value': 0.6221959463893278,
                                                                   'valid_predictions': 2274},
                                        'mean_absolute_percentage_error': {   'metric_value': 0.04375820062480316,
                                                                              'valid_predictions': 2274},
                                        'r2_

Get details for the full `usna_cn2_sm` regression task

In [26]:
task = task_api.get_task("regression.usna_cn2_sm.full.Cn2_3m")

In [ ]:
task = task_api.get_task("regression.usna_cn2_sm.full.Cn2_3m")

In [27]:
type(task)

otb.tasks.tasks.RegressionTask

In [28]:
task.get_all_info()

{'description': 'Regression task for USNA Cn2 small data, where the last 15 days are set aside for evaluation.',
 'description_long': "This dataset evaluates regression approaches for predicting the extent of optical turbulence, as measured by Cn2 at an elevation of 3m. Optical turbulence on data collected at the United States Naval Academy, across the Severn River between Santee Basin and the Waterfront Readiness Center. This dataset contains measurements between 01 June 2021 and 31 August 2021, inclusive. Meterological and Oceanographic measurements are collated using the scintillometer's internal clock at a 6-minute frequency. Prediction accuracy is evaluated  under the root-mean square error metric.",
 'ds_name': 'usna_cn2_sm',
 'obs_tz': 'US/Eastern',
 'obs_lat': 38.98,
 'obs_lon': -76.48,
 'train_idx': ['0:14640'],
 'test_idx': ['14640:18000'],
 'val_idx': ['18000:22081'],
 'dropna': False,
 'log_transform': True,
 'eval_metrics': ['root_mean_square_error',
  'r2_score',
  'mean_

Get the training data

In [29]:
X_train, y_train = task.get_train_data(data_type="pd")

Define or train your model

In [32]:
model = AWTModel(
    name="awt_model",
    air_temperature_col_name="T_5m",
    water_temperature_col_name="T_0m"
)

task.evaluate_model(predict_call=model.predict, x_transforms=None, x_transform_kwargs=None)

In [37]:
model = PersistanceRegressionModel(
    name="mean_model",
)

model.train(X_train, y_train)

task.evaluate_model(predict_call=model.predict, x_transforms=None, x_transform_kwargs=None)

{'root_mean_square_error': {'metric_value': 0.4860083120440881,
  'valid_predictions': 4080},
 'r2_score': {'metric_value': -0.10302263599092476, 'valid_predictions': 4080},
 'mean_absolute_error': {'metric_value': 0.3903624308911522,
  'valid_predictions': 4080},
 'mean_absolute_percentage_error': {'metric_value': 0.027622987088901995,
  'valid_predictions': 4080}}

In [ ]:
model = PersistanceRegressionModel(
    name="mean_model",
)

model.train(X_train, y_train)

task.evaluate_model(predict_call=model.predict, x_transforms=None, x_transform_kwargs=None)

Get details for the `mlo_cn2` forecasting task, with missing values dropped

In [38]:
task = task_api.get_task("forecasting.mlo_cn2.dropna.Cn2_15m")

In [39]:
type(task)

otb.tasks.tasks.ForecastingTask

In [40]:
task.get_all_info()

{'description': 'Forecasting task for MLO Cn2 data, where the last 12 days are set aside for evaluation',
 'description_long': 'This dataset evaluates forecasting approaches for predicting the extent of optical turbulence, as measured by Cn2 at an elevation of 15m. Optical turbulence on data collected at the Mauna Loa Solar Observatory between 27 July 2006 and 8 August 2006, inclusive, are used to evaluate prediction accuracy for measured Cn2 one observation (6 minutes) in the future. The forecasting task makes the last 4 observations of meterological parameters and measured Cn2 available as inputs to forecasting models.',
 'ds_name': 'mlo_cn2',
 'obs_tz': 'US/Hawaii',
 'obs_lat': 19.53,
 'obs_lon': -155.57,
 'train_idx': ['0:8367'],
 'test_idx': ['8367:10367'],
 'val_idx': ['10367:13943'],
 'dropna': True,
 'log_transform': True,
 'eval_metrics': ['root_mean_square_error',
  'r2_score',
  'mean_absolute_error',
  'mean_absolute_percentage_error'],
 'window_size': 4,
 'forecast_horizon

Get the training data

In [41]:
X_train, y_train = task.get_train_data(data_type="pd")

In [42]:
X_train, y_train = task.prepare_forecasting_data(X_train, y_train)

Train your model

In [44]:
model = lgb.LGBMRegressor(random_state=2020)

In [45]:
model.fit(X_train, y_train)

LGBMRegressor(random_state=2020)

Evaluate your model

In [64]:
task.evaluate_model(predict_call=model.predict, x_transforms=None, x_transform_kwargs=None)

{'root_mean_square_error': (0.66010934, 2444),
 'r2_score': (-1.5093609997407498e-06, 2444),
 'mean_absolute_error': (0.5284559, 2444),
 'mean_absolute_percentage_error': (0.03816938, 2444)}

In [46]:
model = PersistanceForecastingModel(
    name="mean_window_model",
    target_name=task.get_target_name(),
)

task.evaluate_model(predict_call=model.predict, x_transforms=None, x_transform_kwargs=None)

{'root_mean_square_error': {'metric_value': 0.2779041826725006,
  'valid_predictions': 2444},
 'r2_score': {'metric_value': 0.8227610719548378, 'valid_predictions': 2444},
 'mean_absolute_error': {'metric_value': 0.196153461933136,
  'valid_predictions': 2444},
 'mean_absolute_percentage_error': {'metric_value': 0.014085889793932438,
  'valid_predictions': 2444}}

Compare against benchmarks

Get details for the `usna_cn2_sm` forecasting task

In [47]:
task = task_api.get_task("forecasting.usna_cn2_sm.full.Cn2_3m")

In [48]:
type(task)

otb.tasks.tasks.ForecastingTask

In [49]:
task.get_all_info()

{'description': 'Forecasting task for USNA Cn2 small data, where the last 15 days are set aside for evaluation.',
 'description_long': "This dataset evaluates forecasting approaches for predicting the extent of optical turbulence, as measured by Cn2 at an elevation of 3m. Optical turbulence on data collected at the United States Naval Academy, across the Severn River between Santee Basin and the Waterfront Readiness Center. This dataset contains measurements between 01 June 2021 and 31 August 2021, inclusive. Meterological and Oceanographic measurements are collated using the scintillometer's internal clock at a 6-minute frequency. Prediction accuracy is evaluated  under the root-mean square error metric.",
 'ds_name': 'usna_cn2_sm',
 'obs_tz': 'US/Eastern',
 'obs_lat': 38.98,
 'obs_lon': -76.48,
 'train_idx': ['0:14640'],
 'test_idx': ['14640:18000'],
 'val_idx': ['18000:22081'],
 'dropna': False,
 'log_transform': True,
 'eval_metrics': ['root_mean_square_error',
  'r2_score',
  'mea

Get the training data

In [50]:
X_train, y_train = task.get_train_data(data_type="pd")

In [51]:
X_train, y_train = task.prepare_forecasting_data(X_train, y_train)

Train your model

In [52]:
model = lgb.LGBMRegressor(random_state=2020)

In [53]:
model.fit(X_train, y_train)

LGBMRegressor(random_state=2020)

Evaluate your model

In [54]:
task.evaluate_model(predict_call=model.predict, x_transforms=None, x_transform_kwargs=None)

{'root_mean_square_error': {'metric_value': 0.16888039580056738,
  'valid_predictions': 4073},
 'r2_score': {'metric_value': 0.8665714886554514, 'valid_predictions': 4073},
 'mean_absolute_error': {'metric_value': 0.08753583133425404,
  'valid_predictions': 4073},
 'mean_absolute_percentage_error': {'metric_value': 0.00621424978647618,
  'valid_predictions': 4073}}

Compare against benchmarks